In [10]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim
import json
# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
# create a list of words
from os import listdir
from os.path import isfile, join

# alter this path to your local environment
mypath = 'data/election/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [69]:
len(onlyfiles)

8

In [67]:
all_tweets = []

for f in onlyfiles:
    full_name = mypath+'/'+f

    with open(full_name, 'r') as f:
        data = json.load(f)
        all_tweets.extend(data)

In [72]:
raw_df_all_tweet = pd.DataFrame(all_tweets)

In [85]:
df_raw_df_all_tweet = raw_df_all_tweet[['created_at','entities','favorite_count','favorited',
                                          'filter_level','id', 'id_str', 'in_reply_to_screen_name',
                                          'in_reply_to_status_id', 'in_reply_to_status_id_str',
                                          'in_reply_to_user_id', 'in_reply_to_user_id_str', 
                                          'is_quote_status','lang','possibly_sensitive', 'quoted_status',
                                          'quoted_status_id', 'quoted_status_id_str','retweeted', 
                                          'retweeted_status', 'source', 'text', 'timestamp_ms','truncated', 'user']]

df_all_tweet = raw_df_all_tweets[['id','text']]

In [83]:
len(raw_df_all_tweet)

796460

In [145]:
geo_detected = raw_df_all_tweet[raw_df_all_tweet['geo'].notnull()].reset_index(drop=True)
len(geo_detected)

399

In [216]:
from folium.plugins import MarkerCluster
import folium
map1  = folium.Map(location=[52.6667, 4.85], zoom_start=2)
marker_cluster = MarkerCluster().add_to(map1)

for point in range(0, len(geo_detected)-250):
     folium.Marker(location=geo_detected['geo'][point]['coordinates'], 
                   popup=folium.Popup(folium.Html(geo_detected['user'][point]['name']+':', 
                                                  script=True),max_width=2650)).add_to(marker_cluster)
map1


In [79]:
all_sensitive_tweet = (raw_df_all_tweet[raw_df_all_tweet['possibly_sensitive'] == True])

In [ ]:
len(all_sensitive_tweet)

In [88]:
filtered_all_sensitive_tweet=all_sensitive_tweet[['id','text']]

In [90]:
filtered_all_sensitive_tweet['pre_clean_len'] = [len(t) for t in filtered_all_sensitive_tweet.text]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [92]:
from pprint import pprint
data_dict = {
    'text':{
        'type':filtered_all_sensitive_tweet.text.dtype,
        'description':'tweet text'
    },
    'pre_clean_len':{
        'type':filtered_all_sensitive_tweet.pre_clean_len.dtype,
        'description':'Length of the tweet before cleaning'
    },
    'dataset_shape':filtered_all_sensitive_tweet.shape
}
pprint(data_dict)

{'dataset_shape': (10193, 3),
 'pre_clean_len': {'description': 'Length of the tweet before cleaning',
                   'type': dtype('int64')},
 'text': {'description': 'tweet text', 'type': dtype('O')}}


In [98]:
len(filtered_all_sensitive_tweet[filtered_all_sensitive_tweet.pre_clean_len > 140])

35

In [110]:
len(filtered_all_sensitive_tweet)

10193

In [131]:
import bs4
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))
www_pat = r'www.[^ ]+'

def tweet_cleaner(text):
    soup =  bs4.BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom_removed = souped
    stripped = re.sub(combined_pat, '', bom_removed)
    stripped = re.sub(www_pat, '', stripped)
    lower_case = stripped.lower()
    letters_only = re.sub("[^a-zA-Z]", " ", lower_case)
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1]
    return (" ".join(words)).strip()

In [132]:
filtered_all_sensitive_tweet = filtered_all_sensitive_tweet.reset_index(drop=True)

In [217]:
clean_tweet_texts = []
for i in range(0,len(filtered_all_sensitive_tweet)):                                                                  
    clean_tweet_texts.append(tweet_cleaner(filtered_all_sensitive_tweet['text'][i]))

C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.

C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.

C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.

C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.

C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.

...

C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.

C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.

C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.

C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.

C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.

In [218]:
filtered_all_sensitive_tweet['text']=clean_tweet_texts

In [220]:
filtered_all_sensitive_tweet.head()

,id,text,pre_clean_len
0,962081686107447297,die is de grootste lul van nederland en ik ga ...,140
1,962083649683443712,oops news de zwetskoning baudet discrimineert ...,134
2,962096369048735744,in weesp mag je in de raad niet eens vragen na...,140
3,962097266898153472,pechtold ging echt veel en veel te ver lijkt m...,83
4,962101570698711040,sylviacool is lekker bezig met campagne voeren...,140


In [221]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(filtered_all_sensitive_tweet['text']))

In [224]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('nl', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

['is grootste lul nederland ga zorgen zwarte poen uit veilingverleden zijn', 'oops zwetskoning baudet discrimineert geloof geslacht geaardheid ras']


In [225]:
print(data_words[:2])
print(data_lemmatized[:2])

[['die', 'is', 'de', 'grootste', 'lul', 'van', 'nederland', 'en', 'ik', 'ga', 'zorgen', 'dat', 'al', 'die', 'zwarte', 'poen', 'uit', 'dat', 'veilingverleden', 'zijn'], ['oops', 'news', 'de', 'zwetskoning', 'baudet', 'discrimineert', 'op', 'basis', 'van', 'geloof', 'geslacht', 'geaardheid', 'en', 'ras']]
['is grootste lul nederland ga zorgen zwarte poen uit veilingverleden zijn', 'oops zwetskoning baudet discrimineert geloof geslacht geaardheid ras']


In [226]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [227]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  0.395921505639 %


In [ ]:
topics = 10
# Build LDA Model
lda_model = LatentDirichletAllocation(n_topics=topics,               # Number of topics
                                      max_iter=20,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [ ]:
# Log Likelihood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
pprint(lda_model.get_params())

In [ ]:
best_lda_model = lda_model
data=text_list

In [ ]:
# Create Document - Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i+1) for i in range(topics)]

# index names
docnames = ["Doc" + str(i) for i in range(len(data))]


In [ ]:
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=id_list)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic+1


# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

In [ ]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
panel

In [15]:
portion_tweet = []
full_name = mypath+'/'+onlyfiles[0]
data = json.load(open(full_name, 'r'))
portion_tweet.extend(data)

raw_df_portion_tweet = pd.DataFrame(portion_tweet)

df_raw_df_portion_tweet = raw_df_portion_tweet[['created_at','entities','favorite_count','favorited',
                                          'filter_level','id', 'id_str', 'in_reply_to_screen_name',
                                          'in_reply_to_status_id', 'in_reply_to_status_id_str',
                                          'in_reply_to_user_id', 'in_reply_to_user_id_str', 
                                          'is_quote_status','lang','possibly_sensitive', 'quoted_status',
                                          'quoted_status_id', 'quoted_status_id_str','retweeted', 
                                          'retweeted_status', 'source', 'text', 'timestamp_ms','truncated', 'user']]

df_portion_tweet = raw_df_portion_tweets[['id','text']]

In [35]:
raw_df_portion_tweet.columns

Index(['contributors', 'coordinates', 'created_at', 'display_text_range',
       'entities', 'extended_entities', 'extended_tweet', 'favorite_count',
       'favorited', 'filter_level', 'geo', 'id', 'id_str',
       'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place',
       'possibly_sensitive', 'quote_count', 'quoted_status',
       'quoted_status_id', 'quoted_status_id_str', 'reply_count',
       'retweet_count', 'retweeted', 'retweeted_status', 'source', 'text',
       'timestamp_ms', 'truncated', 'user', 'withheld_in_countries'],
      dtype='object')

In [65]:
geo_detected = raw_df_portion_tweet[raw_df_portion_tweet['geo'].notnull()]

In [34]:
raw_df_portion_tweet['possibly_sensitive'].value_counts()

False    23453
True       909
Name: possibly_sensitive, dtype: int64

In [77]:
len(raw_df_portion_tweet[raw_df_portion_tweet['possibly_sensitive'] == True])

909